<a href="https://colab.research.google.com/github/TAEO2474/python-dev/blob/main/410_%EB%8D%B0%EC%9D%B4%ED%84%B0%ED%94%84%EB%A0%88%EC%9E%84%EC%9D%98_%EB%8B%A4%EC%96%91%ED%95%9C_%EC%9D%91%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ① Pandas 함수 매핑  
Pandas 함수매핑이란?Pandas에서 데이터프레임이나 시리즈의 각 요소 또는 컬럼 전체에 함수를 적용하는 방법을 의미한다. 이 기능은 데이터 전처리, 변환, 파생 변수 생성 등에 매우 자주 사용된다.

Pandas 함수 매핑

| 함수           | 대상               | 설명                 | 예시                                                         |
| ------------ | ---------------- | ------------------ | ---------------------------------------------------------- |
| `map()`      | Series           | 요소별 함수 적용 (1차원)    | `df['col'].map(lambda x: x*2)`                             |
| `apply()`    | Series/DataFrame | 행/열 또는 요소별 함수 적용   | `df['col'].apply(np.sqrt)` <br> `df.apply(np.sum, axis=1)` |
| `replace()`  | Series/DataFrame | 값 치환               | `df['col'].replace({'Male': 1, 'Female': 0})`              |
| `astype()`   | Series/DataFrame | 데이터 타입 변경          | `df['col'].astype(float)`                                  |
| `pipe()`     | DataFrame        | 함수 체인 처리용          | `df.pipe(my_custom_func)`                                  |


In [ ]:
import pandas as pd
import numpy as np

df = pd.DataFrame({
    'name': ['Alice', 'Bob', 'John'],
    'score': [88, 92, 95],
    'point': [3,5,4],
    'gender': ['F', 'M', 'M']
})


def add_bonus(n):
    return n + 5

def add_bonus2(n, m):
  return n+m

def add_plus(n):
  return n+1

df['bonus'] = df['score'].map(add_bonus)

df['bonus2'] = df['score'].apply(add_bonus2, m=5)

df['bonus3'] = df['score'].apply(lambda n, m : n + m, m=5)

df[['score_2','plus']] = df[['score','point']].apply(add_plus)

# map: 각 요소에 함수 적용 (Series 전용)
df['name_length'] = df['name'].map(len)

# apply: 시리즈나 행/열에 함수 적용
df['score_sqrt'] = df['score'].apply(np.sqrt)

# apply: 데이터프레임의 각 요소에 적용
df_upper = df[['name']].map(str.upper)

# replace: 특정 값 바꾸기
df['gender_num'] = df['gender'].replace({'F': 0, 'M': 1})

# astype: 타입 변환
df['gender_num'] = df['gender_num'].astype('int')

# pipe: 사용자 정의 함수 체인
def add_bonus(df):
    df['bonus'] = df['score'] + 5
    return df

df = df.pipe(add_bonus)
print(df)

    name  score  point gender  bonus  bonus2  bonus3  score_2  plus  \
0  Alice     88      3      F     93      93      93       89     4   
1    Bob     92      5      M     97      97      97       93     6   
2   John     95      4      M    100     100     100       96     5   

   name_length  score_sqrt  gender_num  
0            5    9.380832           0  
1            3    9.591663           1  
2            4    9.746794           1  


/tmp/ipython-input-1-2920326902.py:39: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['gender_num'] = df['gender'].replace({'F': 0, 'M': 1})


In [ ]:
# 각 열에 대해 최대값, 최소값, 평균, 중간값을 계산하는 함수
def calculate_stats(col):
    max_val = col.max()
    min_val = col.min()
    mean_val = col.mean()
    median_val = col.median()

    # 새로운 시리즈 반환
    return pd.Series([max_val, min_val, mean_val, median_val], index=['Max', 'Min', 'Mean', 'Median'])

# 각 열에 calculate_stats 함수 적용 (데이터프레임을 반환)
df_number = df.select_dtypes(include=[np.number])
print(df.info())

result_df = df_number.apply(calculate_stats, axis=0)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         3 non-null      object 
 1   score        3 non-null      int64  
 2   point        3 non-null      int64  
 3   gender       3 non-null      object 
 4   bonus        3 non-null      int64  
 5   bonus2       3 non-null      int64  
 6   bonus3       3 non-null      int64  
 7   score_2      3 non-null      int64  
 8   plus         3 non-null      int64  
 9   name_length  3 non-null      int64  
 10  score_sqrt   3 non-null      float64
 11  gender_num   3 non-null      int64  
dtypes: float64(1), int64(9), object(2)
memory usage: 420.0+ bytes
None


In [ ]:
result_sr = df_number.apply(lambda x : x.max()- x.min(), axis=0)
print(result_sr)

score          7.000000
point          2.000000
bonus          7.000000
bonus2         7.000000
bonus3         7.000000
score_2        7.000000
plus           2.000000
name_length    2.000000
score_sqrt     0.365963
gender_num     1.000000
dtype: float64


# ✅ 성적을 이용한 면접자들의 합계, 평균 구하기

In [ ]:
# 출력할때 컬럼이 한줄로 보이도록 설정
pd.set_option('display.expand_frame_repr', False)

In [ ]:
import pandas as pd

data = {
    'student_id': ['S001', 'S002', 'S003', 'S004', 'S005'],
    'major_score': [88, 75, 92, 81, 67],         # 전공 점수
    'interview_score': [85, 70, 90, 78, 74],     # 면접 점수
    'english_score': [80, 60, 85, 75, 65]        # 영어 점수
}

df = pd.DataFrame(data)
print(df)

df_num=df.select_dtypes(include=[np.number])
print(df_num)

df_row_sum = df_num.apply(lambda x: sum(x), axis=1)
df['total'] =df_row_sum
print(df)

#print(df_num.count(axis=1))

df_row_avg = df['total'] / df_num.count(axis=1)
#print(df_row_avg)
df['avg'] = df_row_avg
print(df)

df['result'] = df['avg'].map(lambda x : "합격"  if x >=80 else "불합격")
print(df)

  student_id  major_score  interview_score  english_score
0       S001           88               85             80
1       S002           75               70             60
2       S003           92               90             85
3       S004           81               78             75
4       S005           67               74             65
   major_score  interview_score  english_score
0           88               85             80
1           75               70             60
2           92               90             85
3           81               78             75
4           67               74             65
  student_id  major_score  interview_score  english_score  total
0       S001           88               85             80    253
1       S002           75               70             60    205
2       S003           92               90             85    267
3       S004           81               78             75    234
4       S005           67               74   

# ✅  pandas.DataFrame.pipe() 함수
-  일반적으로 DataFrame을 리턴하지만, 반환값은 반드시 DataFrame일 필요는 없다. 함수에 따라 Series, 스칼라값, 리스트 등 다양한 타입이 리턴될 수 있다.
- 쉽게 말해, DataFrame을 어떤 함수에 넣어주고, 그 함수의 결과를 그대로 받아오는 도구야.

| 함수 이름        | 리턴값 타입      | pipe 이후 연속 처리 가능 여부 |
| ------------ | ----------- | ------------------- |
| DataFrame 리턴 | `DataFrame` | 가능                |
| 평균 계산 함수     | `Series`    | 불가능                |
| 총합 계산 함수     | `int/float` | 불가능                |
| 리스트 추출 함수    | `list`      | 불가능                |

In [ ]:
import pandas as pd

df = pd.DataFrame({
    'student_id': ['S001', 'S002'],
    'major_score': [88, 75],
    'interview_score': [85, 70],
    'english_score': [80, 60]
})

# 함수 정의들
def get_dataframe(df):
    return df[['student_id', 'major_score']]

def get_series(df):
    return df.mean(numeric_only=True)

def get_scalar(df):
    return df[['major_score', 'interview_score', 'english_score']].sum().sum()

def get_list(df):
    return df['student_id'].tolist()

# 적용
print("DataFrame 리턴:", df.pipe(get_dataframe), '\n')
print("Series 리턴:\n", df.pipe(get_series), '\n')
print("Scalar 리턴:", df.pipe(get_scalar), '\n')
print("List 리턴:", df.pipe(get_list), '\n')

print('---------------------')
print("DataFrame 리턴:", df.pipe(get_dataframe).pipe(get_series), '\n')

DataFrame 리턴:   student_id  major_score
0       S001           88
1       S002           75 

Series 리턴:
 major_score        81.5
interview_score    77.5
english_score      70.0
dtype: float64 

Scalar 리턴: 458 

List 리턴: ['S001', 'S002'] 

---------------------
DataFrame 리턴: major_score    81.5
dtype: float64 



# ✅ 체인 기법이란?
- DataFrame이나 Series에서 메서드들을 점(.)으로 계속 이어서 연결하는 방식
- 👉 데이터를 가공하거나 분석할 때 코드를 깔끔하고 직관적으로 표현할 수 있어요.

In [ ]:
import pandas as pd

df = pd.DataFrame({
    'name': ['Tom', 'Jane', 'Alice', 'Bob'],
    'score': [80, 90, 85, 70]
})

# 체인 기법 사용 예
df_result = (
    df
    .query("score >= 80")
    .sort_values(by="score", ascending=False)
    .reset_index(drop=True)
)
#  df.query("score >= 80"): 80점 이상 필터링
# .sort_values(...): 점수 내림차순 정렬
# .reset_index(drop=True): 인덱스 초기화

print(df_result)


    name  score
0   Jane     90
1  Alice     85
2    Tom     80


In [ ]:
df.pipe(lambda d: d[d['score'] >= 80]) \
  .sort_values('score') \
  .reset_index(drop=True)

,name,score
0,Tom,80
1,Alice,85
2,Jane,90


## ② 열 **재구성**

열 순서 변경

In [ ]:
import pandas as pd
import numpy as np

data = {
    'student_id': ['S001', 'S002', 'S003', 'S004', 'S005'],
    'major_score': [88, 75, 92, 81, 67],         # 전공 점수
    'interview_score': [85, 70, 90, 78, 74],     # 면접 점수
    'english_score': [80, 60, 85, 75, 65]        # 영어 점수
}

df = pd.DataFrame(data)


df_num = df.select_dtypes(include=[np.number])
print(df_num)
print(type(df_num))               # pandas.core.frame.DataFrame
print(df_num.columns)
print(type(df_num.columns))       # pandas.core.indexes.base.Index

print(df_num.columns.values)
print(type(df_num.columns.values))  # numpy.ndarray
print(list(df_num.columns.values))  # list
print(sorted(list(df_num.columns.values), reverse=False)) # reverse=False 오름차순(default)
columns_sorted = sorted(list(df_num.columns.values))

# 정렬된 숫자형 컬럼 + student_id 열을 뒤에 붙이기
final_columns = ['student_id'] + columns_sorted
df_sorted = df[final_columns]

print(df_sorted)

   major_score  interview_score  english_score
0           88               85             80
1           75               70             60
2           92               90             85
3           81               78             75
4           67               74             65
<class 'pandas.core.frame.DataFrame'>
Index(['major_score', 'interview_score', 'english_score'], dtype='object')
<class 'pandas.core.indexes.base.Index'>
['major_score' 'interview_score' 'english_score']
<class 'numpy.ndarray'>
['major_score', 'interview_score', 'english_score']
['english_score', 'interview_score', 'major_score']
  student_id  english_score  interview_score  major_score
0       S001             80               85           88
1       S002             60               70           75
2       S003             85               90           92
3       S004             75               78           81
4       S005             65               74           67


In [ ]:
import pandas as pd

# 2025년 1월 1일부터 10일간의 일별 시계열
df = pd.DataFrame({
    'variable': pd.date_range(start='2025-01-01', periods=10, freq='D')
})
print(df)


 # split('-', expand=False) default :expand=False 이면 list로 반환
df_split_list = df['variable'].astype('str').str.split('-')
#print(type(df_split_list))
#print(df_split_list)
print(df_split_list.str)
print(type(df_split_list.str)) # StringMethods

print(df_split_list.str.get(0))  # df_split_list.str[0]
print(df_split_list.str.get(1))  # df_split_list.str[1]
print(df_split_list.str.get(2))  # df_split_list.str[2]

df['year'] = df_split_list.str[0]
df['month'] = df_split_list.str[1]
df['day'] = df_split_list.str[2]
print(df)


df_split_multi = df['variable'].astype('str').str.split('-', expand=True)
print(type(df_split_multi))

df['year'] = df_split_multi[0]
df['month'] = df_split_multi[1]
df['day'] = df_split_multi[2]
print(df)

    variable
0 2025-01-01
1 2025-01-02
2 2025-01-03
3 2025-01-04
4 2025-01-05
5 2025-01-06
6 2025-01-07
7 2025-01-08
8 2025-01-09
9 2025-01-10
<class 'pandas.core.strings.accessor.StringMethods'>
0    2025
1    2025
2    2025
3    2025
4    2025
5    2025
6    2025
7    2025
8    2025
9    2025
Name: variable, dtype: object
0    01
1    01
2    01
3    01
4    01
5    01
6    01
7    01
8    01
9    01
Name: variable, dtype: object
0    01
1    02
2    03
3    04
4    05
5    06
6    07
7    08
8    09
9    10
Name: variable, dtype: object
    variable  year month day
0 2025-01-01  2025    01  01
1 2025-01-02  2025    01  02
2 2025-01-03  2025    01  03
3 2025-01-04  2025    01  04
4 2025-01-05  2025    01  05
5 2025-01-06  2025    01  06
6 2025-01-07  2025    01  07
7 2025-01-08  2025    01  08
8 2025-01-09  2025    01  09
9 2025-01-10  2025    01  10
<class 'pandas.core.frame.DataFrame'>
    variable  year month day
0 2025-01-01  2025    01  01
1 2025-01-02  2025    01  02
2 2025-01-

In [ ]:
import pandas as pd

# 2025년 1월 1일부터 10일간의 일별 시계열
df = pd.DataFrame({
    'variable': pd.date_range(start='2025-01-01', periods=10, freq='D')
})
#print(df)



# split('-', expand=False) default :expand=False 이면 list로 반환
df_split_list = df['variable'].astype('str').str.split('-')
#print(type(df['variable'].astype('str').str))  # StringMethods
#print(dir(df['variable'].astype('str').str))


print(df_split_list.str.get(0)) # YEAR
print(df_split_list.str.get(1)) # MONTH
print(df_split_list.str.get(2)) # DATE
print(df_split_list.str.get(3)) # 없음(NONE)
print(df_split_list.astype('str').get(0)) # 0행 년월일 가져오기

0    2025
1    2025
2    2025
3    2025
4    2025
5    2025
6    2025
7    2025
8    2025
9    2025
Name: variable, dtype: object
0    01
1    01
2    01
3    01
4    01
5    01
6    01
7    01
8    01
9    01
Name: variable, dtype: object
0    01
1    02
2    03
3    04
4    05
5    06
6    07
7    08
8    09
9    10
Name: variable, dtype: object
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
5   NaN
6   NaN
7   NaN
8   NaN
9   NaN
Name: variable, dtype: float64
['2025', '01', '01']


In [ ]:
df['year'] = df_split_list.str.get(0)     # df_split_list.str[0]
df['month'] = df_split_list.str.get(1)    # df_split_list.str[1]
df['day'] = df_split_list.str.get(2)      # df_split_list.str[2]

print(df.head())

    variable  year month day
0 2025-01-01  2025    01  01
1 2025-01-02  2025    01  02
2 2025-01-03  2025    01  03
3 2025-01-04  2025    01  04
4 2025-01-05  2025    01  05


In [ ]:
#👉 'variable' 열의 값을 문자열로 바꾼 뒤, '-' 기호 기준으로 나눠서 여러 열로 분리하라
df_split_multi = df['variable'].astype('str').str.split('-',expand=True)
print(type(df_split_multi))
print(df_split_multi.head())

df['year'] = df_split_multi[0]
df['month'] = df_split_multi[1]
df['day'] = df_split_multi[2]
print(df.head())

<class 'pandas.core.frame.DataFrame'>
      0   1   2
0  2025  01  01
1  2025  01  02
2  2025  01  03
3  2025  01  04
4  2025  01  05
    variable  year month day
0 2025-01-01  2025    01  01
1 2025-01-02  2025    01  02
2 2025-01-03  2025    01  03
3 2025-01-04  2025    01  04
4 2025-01-05  2025    01  05


## ③ pandas.groupby() - 그룹연산함수

- 하나 이상의 열을 기준으로 데이터를 묶고, 묶인 각 그룹에 대해 합계, 평균, 개수 등 연산을 적용할 수 있는 함수.
-  데이터를 그룹으로 나누어(분할) 집계, 변환, 필터링하는 함수이고, "분할 → 적용 → 결합(Split-Apply-Combine)" 전략을 기반으로 동작한다.

```
df.groupby(by='컬럼명')
```

##✅  GroupBy 단계

| 단계              | 설명                        |
| --------------- | ------------------------- |
| **Split(분할)**   | 지정한 컬럼을 기준으로 데이터를 그룹으로 나눔 |
| **Apply(적용)**   | 각 그룹에 함수(sum, mean 등)를 적용 |
| **Combine(결합)** | 결과를 하나의 DataFrame으로 결합    |

# ✅ observed 옵션이란?
- 범주형(Categorical) 열을 기준으로 그룹을 나눌 때,
실제로 관측(observed)된 조합만 포함할지,
아니면 모든 가능한 조합을 포함할지를 선택하는 옵션입니다.

```
DataFrame.groupby(by, observed=True)
```
| 옵션                     | 의미                                                             |
| ---------------------- | -------------------------------------------------------------- |
| `observed=False` (기본값) | 범주형 컬럼(group key)의 **모든 가능한 조합**을 결과에 포함함 (심지어 데이터가 없는 조합도 포함) |
| `observed=True`        | **실제로 관측된 조합만** 결과에 포함함 → **속도 개선**, **메모리 절약**                |



In [ ]:
import pandas as pd

# 범주형 데이터 생성
df = pd.DataFrame({
    'department': pd.Categorical(['HR', 'HR','HR','HR', 'IT', 'IT','IT', 'IT'], categories=['HR', 'IT', 'Finance']),
    'gender': pd.Categorical(['M', 'F',  'F','M','M', 'F','F', 'M'], categories=['M', 'F']),
    'score': [80, 70, 100,90,90, 85,95, 88]
})

print("원본 데이터")
print(df)



원본 데이터
  department gender  score
0         HR      M     80
1         HR      F     70
2         HR      F    100
3         HR      M     90
4         IT      M     90
5         IT      F     85
6         IT      F     95
7         IT      M     88


### ③-1 그룹객체 만들기(분할단계)

 observed=False (기본값) → 모든 조합

In [ ]:
# get_group() : grouped 객체에서 특정 객체만을 선택할 수 있다. get_group()함수의 인자는 튜플형태이다
grouped = df.groupby(['department'], observed=True)  #False (객체에는 존재하지 않는 조합의 그룹(값은 NaN)도 포함)
print(type(grouped)) # DataFrameGroupBy
print(grouped)
for key, group in grouped:
  print(f'key: {key}')
  print(f'group: {group}')

IT = grouped.get_group(('IT'))  # 튜플()
print(IT)

 # → 존재하지 않는 조합(예: Finance-F, Finance-M)도 포함됨 (값은 NaN)

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>
key: ('HR',)
group:   department gender  score
0         HR      M     80
1         HR      F     70
2         HR      F    100
3         HR      M     90
key: ('IT',)
group:   department gender  score
4         IT      M     90
5         IT      F     85
6         IT      F     95
7         IT      M     88
  department gender  score
4         IT      M     90
5         IT      F     85
6         IT      F     95
7         IT      M     88


/tmp/ipython-input-45-437323222.py:9: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  IT = grouped.get_group(('IT'))  # 튜플()


In [ ]:
result_false = df.groupby(['department', 'gender'], observed=False)
print(type(result_false))
print(result_false)
for key, group in result_false:
  print(f'key: {key}')
  print(f'group: {group}')

 # → 존재하지 않는 조합(예: Finance-F, Finance-M)도 포함됨 (값은 NaN)

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>
key: ('HR', 'M')
group:   department gender  score
0         HR      M     80
3         HR      M     90
key: ('HR', 'F')
group:   department gender  score
1         HR      F     70
2         HR      F    100
key: ('IT', 'M')
group:   department gender  score
4         IT      M     90
7         IT      M     88
key: ('IT', 'F')
group:   department gender  score
5         IT      F     85
6         IT      F     95


In [ ]:
result_false = df.groupby(['department', 'gender'], observed=False)
print(type(result_false))
print(result_false)
for key, group in result_false:
  print(f'key: {key}')
  print(f'group: {group}')

 # → 존재하지 않는 조합(예: Finance-F, Finance-M)도 포함됨 (값은 NaN)

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>
key: ('HR', 'M')
group:   department gender  score
0         HR      M     80
3         HR      M     90
key: ('HR', 'F')
group:   department gender  score
1         HR      F     70
2         HR      F    100
key: ('IT', 'M')
group:   department gender  score
4         IT      M     90
7         IT      M     88
key: ('IT', 'F')
group:   department gender  score
5         IT      F     85
6         IT      F     95


# observed=True → 실제 관측된 조합만

In [ ]:
result_true = df.groupby(['department'], observed=True)
print(result_true)
for key, group in result_true:
  print(f'key: {key}')
  print(f'group: {group}')

# → 실제 데이터에 존재하는 조합만 출력 (NaN 없음, 더 간결하고 빠름)

key: ('HR',)
group:   department gender  score
0         HR      M     80
1         HR      F     70
2         HR      F    100
3         HR      M     90
key: ('IT',)
group:   department gender  score
4         IT      M     90
5         IT      F     85
6         IT      F     95
7         IT      M     88


### ③-2 그룹 연산 메소드(적용-결합 단계)
#### ③-2-1 집계(Aggregation)
- 그룹객체에 다양한 연산을 적용할 수 있는데 이 과정을 데이터 집계(Aggregation)라고 한다.
- 집계기능을 내장하고 있는 판단스 기본함수에는 mean(), max(), min(), sum(), count(), size(), var(), std(), describe(), info(), first(), last()등이 있다.


<Pandas 주요 통계 및 집계 함수 정리>



| 함수명          | 설명                                                       | 사용 예시                                   | 리턴 형태               |
| ------------ | -------------------------------------------------------- | --------------------------------------- | ------------------- |
| `mean()`     | 평균 (산술 평균)                                               | `df.mean()` or `df.groupby(...).mean()` | Series or DataFrame |
| `max()`      | 최댓값                                                      | `df.max()`                              | Series or DataFrame |
| `min()`      | 최솟값                                                      | `df.min()`                              | Series or DataFrame |
| `sum()`      | 합계                                                       | `df.sum()`                              | Series or DataFrame |
| `count()`    | **NaN 제외한 값 개수**                                         | `df.count()`                            | Series or DataFrame |
| `size()`     | **전체 행 개수** (`groupby()`에서 사용)                           | `df.groupby('col').size()`              | Series              |
| `var()`      | 분산 (기본: sample variance, N-1)                            | `df.var()`                              | Series or DataFrame |
| `std()`      | 표준편차 (기본: sample std)                                    | `df.std()`                              | Series or DataFrame |
| `describe()` | **기초 통계 요약** (count, mean, std, min, 25%, 50%, 75%, max) | `df.describe()`                         | DataFrame           |
| `info()`     | **컬럼 요약 정보, null 수, 타입 등**                               | `df.info()`                             | 콘솔 출력 (None)        |
| `first()`    | 그룹 또는 시계열 내 **첫 번째 행/값**                                 | `df.groupby('col').first()`             | Series or DataFrame |
| `last()`     | 그룹 또는 시계열 내 **마지막 행/값**                                  | `df.groupby('col').last()`              | Series or DataFrame |


주요 비교 정리


| 함수           | NaN 포함 여부 | 그룹 집계용 사용    | 시계열에도 사용 가능 | 전체 통계 제공  |
| ------------ | --------- | ------------ | ----------- | --------- |
| `mean()`     | 제외됨       | O            | O           | X          |
| `count()`    | NaN 제외    | O           | O           | X          |
| `size()`     | NaN 포함    | O (Group 전용) | X            | X          |
| `describe()` | 제외됨       | O           | X            | O         |
| `info()`     | 제외됨       | X           | X            | O (요약 전용) |
| `first()`    | 제외됨       | O            | O           | X          |
| `last()`     | 제외됨       | O           | O          | X          |








#✅  numeric_only=True
- 숫자형 열만 평균 계산에 사용합니다.

- object, bool, datetime 등 숫자가 아닌 열은 무시하고 연산합니다.

- 불필요한 경고 메시지도 피할 수 있어요.

In [ ]:
grouped = df.groupby(['department'], observed=False)
print(type(grouped))
mean_department = grouped.mean(numeric_only=True)
print("observed=False 결과")
print(type(mean_department))
print(mean_department)


 # → 존재하지 않는 조합(예: Finance-F, Finance-M)도 포함됨 (값은 NaN)


<class 'pandas.core.groupby.generic.DataFrameGroupBy'>
observed=False 결과
<class 'pandas.core.frame.DataFrame'>
            score
department       
HR           85.0
IT           89.5
Finance       NaN


In [ ]:
mean_department.head()

,score
department,
HR,85.0
IT,89.5
Finance,NaN


In [ ]:
mean_department.describe()

,score
count,2.000000
mean,87.250000
std,3.181981
min,85.000000
25%,86.125000
50%,87.250000
75%,88.375000
max,89.500000


#✅  as_index=True와 as_index=False 비교

In [ ]:
# as_index=False는 그룹화의 기준 열(department)을 데이터 프레임의 새로운 인덱스로 설정하지 않고 데이터프레임의 열로 유지한다는 의미이다.
# 결과적으로 원본 데이터프레임과 동일한 열 인덱스를 유지한다.
grouped = df.groupby(['department'], observed=False, as_index=True)
print(type(grouped))
print(grouped)
# 고유값의 빈도수 추출
grouped[['department','score']].value_counts()

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>


department  score
HR          70       1
            80       1
            90       1
            100      1
            85       0
            88       0
            95       0
IT          85       1
            88       1
            90       1
            95       1
            70       0
            80       0
            100      0
Finance     70       0
            80       0
            85       0
            88       0
            90       0
            95       0
            100      0
Name: count, dtype: int64

In [ ]:
# as_index=False는 그룹화의 기준 열(department)을 데이터 프레임의 새로운 인덱스로 설정하지 않고 데이터프레임의 열로 유지한다는 의미이다.
# 결과적으로 원본 데이터프레임과 동일한 열 인덱스를 유지한다.
grouped = df.groupby(['department'], observed=False, as_index=False)
print(type(grouped))
print(grouped)
# 고유값의 빈도수 추출
grouped[['department','score']].value_counts()

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>


,department,score,count
0,HR,70,1
1,HR,80,1
2,HR,90,1
3,HR,100,1
4,HR,85,0
5,HR,88,0
6,HR,95,0
7,IT,85,1
8,IT,88,1
9,IT,90,1


In [ ]:
result_false = df.groupby(['department', 'gender'], observed=False, as_index=False).mean(numeric_only=True)
print("observed=False 결과")
print(type(result_false))
print(result_false)

 # → 존재하지 않는 조합(예: Finance-F, Finance-M)도 포함됨 (값은 NaN)

observed=False 결과
<class 'pandas.core.frame.DataFrame'>
  department gender  score
0         HR      M   85.0
1         HR      F   85.0
2         IT      M   89.0
3         IT      F   90.0
4    Finance      M    NaN
5    Finance      F    NaN


In [ ]:
result_false.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   department  6 non-null      category
 1   gender      6 non-null      category
 2   score       4 non-null      float64 
dtypes: category(2), float64(1)
memory usage: 448.0 bytes


In [ ]:
result_true = df.groupby(['department'], observed=True, as_index=False).mean(numeric_only=True)
print("observed=True 결과")
print(type(result_true))
print(result_true)

# → 실제 데이터에 존재하는 조합만 출력 (NaN 없음, 더 간결하고 빠름)

observed=True 결과
<class 'pandas.core.frame.DataFrame'>
  department  score
0         HR   85.0
1         IT   89.5


# ✅ agg() 함수 = aggregate() 함수
- DataFrame이나 Series, 또는 GroupBy 객체에 대해
하나 이상의 함수를 적용할 수 있는 다목적 함수입니다.

- 이름처럼 데이터를 "집계(aggregate)"하는 데 사용됩니다.

In [ ]:
group = df.groupby(['department', 'gender'], observed=False)
department_gender = group.agg('mean', numeric_only=True)
print(department_gender)

                   score
department gender       
HR         M        85.0
           F        85.0
IT         M        89.0
           F        90.0
Finance    M         NaN
           F         NaN


In [ ]:
# 기본함수만 사용할때
group = df.groupby(['department', 'gender'], observed=True)
department_gender = group.agg(['mean', 'max', 'min'], numeric_only=True)
#print(department_gender)

# 사용자 정의 함수 사용할때
def max_minus_min(s):
  return s.max() - s.min()

department_gender = group['score'].agg(['mean', 'max', 'min', max_minus_min])
# 'mean': 평균
# 'max': 최대값
# 'min': 최소값
# max_minus_min: 사용자가 만든 함수

print(department_gender)

                   mean  max  min  max_minus_min
department gender                               
HR         M       85.0   90   80             10
           F       85.0  100   70             30
IT         M       89.0   90   88              2
           F       90.0   95   85             10


# ③-2-2 변환(Transformation)
- 변환은 그룹화된 데이터에 대해 수행되는 연산으로, 원본 데이터와 동일한 행 인덱스와 열 이름을 갖는 데이터프레임을 반환한다는 점에서 그룹별로 연산 결과를 반환하는 집계(Aggregation)와 다르다.
- 즉, 그룹을 나누고 연산을 한다는 점은 두 방식이 같지만, 연산의 결과에 그룹이 포함되는지 여부에 따라서 구분된다.
변환 기능을 내장하고 있는 판단스 기본 함수에는 bfill(), cummcount(), cummax(), cumin(), cumprod(), cumsum(), diff(), ffill(), pct_change(), rank(), shift()등이 있다.

| 함수명            | 설명                                  | 사용 예시                            |
| -------------- | ----------------------------------- | -------------------------------- |
| `bfill()`      | 결측값을 **뒤에서 앞으로** 채움 (Backward Fill) | `df['col'].bfill()`              |
| `ffill()`      | 결측값을 **앞에서 뒤로** 채움 (Forward Fill)   | `df['col'].ffill()`              |
| `cumcount()`   | 그룹별로 행의 \*\*순서(0부터 시작)\*\*를 계산      | `df.groupby('group').cumcount()` |
| `cummax()`     | 누적 **최댓값**                          | `df['col'].cummax()`             |
| `cummin()`     | 누적 **최솟값**                          | `df['col'].cummin()`             |
| `cumprod()`    | 누적 **곱**                            | `df['col'].cumprod()`            |
| `cumsum()`     | 누적 **합**                            | `df['col'].cumsum()`             |
| `diff()`       | 바로 이전 값과의 **차이** (기본: `periods=1`)  | `df['col'].diff()`               |
| `pct_change()` | 바로 이전 값 대비 **백분율 변화율**              | `df['col'].pct_change()`         |
| `rank()`       | 데이터의 **순위** 계산                      | `df['col'].rank()`               |
| `shift()`      | 데이터를 **지정한 기간만큼 이동** (NaN 발생)       | `df['col'].shift(1)`             |


In [ ]:
# HR,IT부서별로 행이 하나씩 추가될때 누적값을 계속 더함
df_1 = df.copy()
grouped = df_1.groupby('department', observed=True)
df_1['score_cumsum'] =   grouped['score'].cumsum()
df_1


,department,gender,score,score_cumsum
0,HR,M,80,80
1,HR,F,70,150
2,HR,F,100,250
3,HR,M,90,340
4,IT,M,90,90
5,IT,F,85,175
6,IT,F,95,270
7,IT,M,88,358


In [ ]:
# gender 컬럼을 그룹으로 설정하고 score 컬럼을 누적계산
df_2 = df.copy()
grouped = df_2.groupby('gender', observed=True)
df_2['score_cumsum'] =   grouped['score'].cumsum()
df_2.sort_values('gender')

,department,gender,score,score_cumsum
0,HR,M,80,80
3,HR,M,90,170
4,IT,M,90,260
7,IT,M,88,348
1,HR,F,70,70
2,HR,F,100,170
5,IT,F,85,255
6,IT,F,95,350


In [ ]:
df_3 = df.copy()
grouped = df_3.groupby(['department','gender'], observed=True)
df_3['score_cumsum'] =   grouped['score'].cumsum()
df_3


,department,gender,score,score_cumsum
0,HR,M,80,80
1,HR,F,70,70
2,HR,F,100,170
3,HR,M,90,170
4,IT,M,90,90
5,IT,F,85,85
6,IT,F,95,180
7,IT,M,88,178


# transform()메소드
-  그룹 객체에 대해 변환 연산을 수행하고 원본 데이터프레임과 같은 형태로 반환한다. 이 메소드는 cumsum() 메소드와 같은 내장 변한 함수나 집계함수의 문자열 별칭을 인자로 받을 수 있으며, 이러한 함수를 그룹별 데이터에 적용하여 변환된 결과를 데이터프레임으로 생성한다.
```
group객체.transform(매핑함수)
```

In [ ]:
df_4 = df.copy()
grouped = df_4.groupby(['department'], observed=True)
df_4['score_cumsum'] =   grouped['score'].transform('cumsum')
print(df_4)

  department gender  score  score_cumsum
0         HR      M     80            80
1         HR      F     70           150
2         HR      F    100           250
3         HR      M     90           340
4         IT      M     90            90
5         IT      F     85           175
6         IT      F     95           270
7         IT      M     88           358


In [ ]:
def  z_score(x):  # 평균0, 표준편차1
    return (x - x.mean()) / x.std()

df_5 = df.copy()
grouped = df_5.groupby(['department'], observed=True)
df_5['score_zscore'] =   grouped['score'].transform(z_score)
print(df_5)

  department gender  score  score_zscore
0         HR      M     80     -0.387298
1         HR      F     70     -1.161895
2         HR      F    100      1.161895
3         HR      M     90      0.387298
4         IT      M     90      0.118958
5         IT      F     85     -1.070620
6         IT      F     95      1.308535
7         IT      M     88     -0.356873


# ③-2-3 filter( ) 메소드
- 필터링(Filtration)은 원본  데이터프레임의 부분 집합 형태의 새로운 데이터프레임을 생성하는 그룹 연산이다. 원본 데이터프레임 중에서 특정한 조건을 만족하는 행 데이터를 추출하여 반환한다.
- 이와 같은 필터링 연산을 수행하는 내장 필터링 메소드에는
   -  head(),
   -  nth(),
   -  tail()
   

In [ ]:
grouped = df.groupby(['department'], observed=True)
# 그룹별로 첫 2행을 확인
df_filtered = grouped.head(2)
print(df_filtered)

  department gender  score
0         HR      M     80
1         HR      F     70
4         IT      M     90
5         IT      F     85


In [ ]:
grouped = df.groupby(['department'], observed=True)
# nth(1) 메소드를 사용하여 각 그룹의 두 번째 행(인덱스 1)을 필터링하여 선택한다.
df_filtered = grouped.nth(1)
print(df_filtered)

  department gender  score
1         HR      F     70
5         IT      F     85


In [ ]:
grouped =df.groupby(['department'], observed=True)
df_filtered = grouped.mean('score')
print(df_filtered)

            score
department       
HR           85.0
IT           89.5


In [ ]:
grouped =df.groupby(['department'], observed=True)
# 내장 필터링 함수를 직접 사용하지 않고 그룹 객체에 filter()메소드를 적용하는 방식
# filter()메소드에 조건식을 가진 함수를 전달하면 조건이 True인 그룹만을 필터링하여 새로운 데이터프레임으로 반환한다.
df_filtered = grouped.filter(lambda x: x['score'].mean() >= 86)
print(df_filtered)

  department gender  score
4         IT      M     90
5         IT      F     85
6         IT      F     95
7         IT      M     88


In [ ]:
grouped =df.groupby(['department'], observed=True)
# 내장 필터링 함수를 직접 사용하지 않고 그룹 객체에 filter()메소드를 적용하는 방식
# filter()메소드에 조건식을 가진 함수를 전달하면 조건이 True인 그룹만을 필터링하여 새로운 데이터프레임으로 반환한다.

#"각 그룹의 'score' 평균이 86 이상인 그룹만 필터링한다."
df_filtered = grouped.filter(lambda x: x['score'].mean() >= 86)
df_filtered

,department,gender,score
4,IT,M,90
5,IT,F,85
6,IT,F,95
7,IT,M,88


#  ③-2-4 그룹 객체에 함수 매핑

# apply()함수
  사용자 정의 함수를 사용하여 그룹 객체에 적용하여 복잡하고 다양한 연산을 처리할 수 있다.

In [ ]:
grouped = df.groupby(['department'], observed=True)
df_applied = grouped.apply(lambda x: x['score'].mean())
print(df_applied)

department
HR    85.0
IT    89.5
dtype: float64


/tmp/ipython-input-87-2474267926.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_applied = grouped.apply(lambda x: x['score'].mean())


# ④ multi.index (멀티인덱스)
- Pandas에서 하나의 축(행 또는 열)에 두 개 이상의 인덱스 레벨을 갖는 구조를 말해요.
-즉, 계층적으로 구성된 인덱스

In [ ]:
# df.groupby()을 이용한 멀티인덱스 생성
grouped = df.groupby(['department', 'gender'], observed=True)
sum_score = grouped['score'].sum()
print(type(sum_score.index))
print(sum_score)

<class 'pandas.core.indexes.multi.MultiIndex'>
department  gender
HR          M         170
            F         170
IT          M         178
            F         180
Name: score, dtype: int64


In [ ]:
# set_index()로 멀티인덱스 만들기
df = pd.DataFrame({
    '지역': ['서울', '서울', '부산'],
    '도시': ['강남', '마포', '해운대'],
    '매출': [100, 80, 95]
})

df_multi = df.set_index(['지역', '도시'])
print(type(df_multi.index))
df_multi


<class 'pandas.core.indexes.multi.MultiIndex'>


매출
지역 도시      
서울 강남   100
   마포    80
부산 해운대   95

In [ ]:
import pandas as pd

#이 상태에서 '도시', '구'는 그냥 열입니다.
arrays = [
    ['서울', '서울', '부산', '부산'],
    ['강남구', '마포구', '해운대구', '중구']
]

# 이제 도시,구 ->  멀티인덱스(행)로 바뀜
index = pd.MultiIndex.from_arrays(arrays, names=('도시', '구'))
data = pd.Series([100, 200, 150, 180], index=index)
data

도시  구   
서울  강남구     100
    마포구     200
부산  해운대구    150
    중구      180
dtype: int64

In [ ]:
# pd.MultiIndex.from_arrays(arrays, names=None)
'''
| 파라미터     | 설명                                   |
| -------- | ------------------------------------ |
| `arrays` | 리스트나 튜플들의 리스트 (각 배열은 하나의 인덱스 레벨을 의미) |
| `names`  | 인덱스 수준(level)에 부여할 이름들               |

- 각 배열은 동일한 길이여야 함
- 배열 순서가 인덱스 레벨의 순서가 됨
'''

arrays = [
    ['서울', '서울', '부산', '부산'],
    ['강남', '마포', '해운대', '서면']
]

index = pd.MultiIndex.from_arrays(arrays, names=['지역', '도시'])

df = pd.DataFrame({'매출': [100, 80, 95, 85]}, index=index)
print(type(df.index))
df

<class 'pandas.core.indexes.multi.MultiIndex'>


매출
지역 도시      
서울 강남   100
   마포    80
부산 해운대   95
   서면    85

In [ ]:
# pd.MultiIndex.from_tuples()

index = pd.MultiIndex.from_tuples([('서울', '강남'), ('서울', '마포'), ('부산', '해운대')],
                                  names=['지역', '도시'])
df = pd.DataFrame({'매출': [100, 80, 95]}, index=index)

print(type(df.index))
df

<class 'pandas.core.indexes.multi.MultiIndex'>


매출
지역 도시      
서울 강남   100
   마포    80
부산 해운대   95

In [ ]:
# pd.MultiIndex.from_product() – 계층적 조합 생성

mi = pd.MultiIndex.from_product([['2023', '2024'], ['Q1', 'Q2']], names=['연도', '분기'])
df = pd.DataFrame({'매출': [120, 130, 140, 150]}, index=mi)
print(type(df.index))
print(df.index)

df

<class 'pandas.core.indexes.multi.MultiIndex'>
MultiIndex([('2023', 'Q1'),
            ('2023', 'Q2'),
            ('2024', 'Q1'),
            ('2024', 'Q2')],
           names=['연도', '분기'])


매출
연도   분기     
2023 Q1  120
     Q2  130
2024 Q1  140
     Q2  150

In [ ]:
# 멀티인덱스 행/열 접근 방법
print("특정 위치 접근: 2023년 Q1")
print(df.loc[('2023', 'Q1')])

print("특정 인덱스 레벨 슬라이싱: 2023년 전체")
idx = pd.IndexSlice
df.loc[idx['2023', :], :]

특정 위치 접근: 2023년 Q1
매출    120
Name: (2023, Q1), dtype: int64
특정 인덱스 레벨 슬라이싱: 2023년 전체


매출
연도   분기     
2023 Q1  120
     Q2  130

In [ ]:
# 멀티인덱스 해제 (reset_index())
df_reset = df.reset_index()
print(type(df_reset.index))
print(df_reset)
df_reset.index.name = None # 인덱스 이름 삭제
print(df_reset)

<class 'pandas.core.indexes.range.RangeIndex'>
     연도  분기   매출
0  2023  Q1  120
1  2023  Q2  130
2  2024  Q1  140
3  2024  Q2  150
     연도  분기   매출
0  2023  Q1  120
1  2023  Q2  130
2  2024  Q1  140
3  2024  Q2  150


In [ ]:
# 멀티인덱스 정렬의 두 가지 방식


index = pd.MultiIndex.from_tuples([
    ('서울', '강남'),
    ('서울', '마포'),
    ('부산', '서면'),
    ('부산', '해운대')
], names=['지역', '지점'])

df = pd.DataFrame({'매출': [100, 80, 95, 85]}, index=index)
print(df)

# 인덱스 기준 정렬 (sort_index())
print('===인덱스 기준 정렬 (sort_index())==')
df_sorted = df.sort_index()
print(df_sorted)

#  특정 인덱스 레벨 기준 정렬
print('== 특정 인덱스 레벨 기준 정렬==')
# '지점' 기준만 정렬 (level=1)
df_sorted = df.sort_index(level='지점')
print(df_sorted)

# 내림차순 정렬
print('=====내림차순 정렬======')
df_sorted = df.sort_index(ascending=False)
print(df_sorted)

# 값 기준 정렬 (sort_values())
print('===값 기준 정렬 (sort_values())===')
df_sorted = df.sort_values(by='매출', ascending=False)
print(df_sorted)

#인덱스를 컬럼으로 풀고 정렬 후 다시 멀티인덱스로
print('===인덱스를 컬럼으로 풀고 정렬 후 다시 멀티인덱스로===')
df_reset = df.reset_index()
df_sorted = df_reset.sort_values(by='매출', ascending=False)
df_final = df_sorted.set_index(['지역', '지점'])
print(df_final)


         매출
지역 지점      
서울 강남   100
   마포    80
부산 서면    95
   해운대   85
===인덱스 기준 정렬 (sort_index())==
         매출
지역 지점      
부산 서면    95
   해운대   85
서울 강남   100
   마포    80
== 특정 인덱스 레벨 기준 정렬==
         매출
지역 지점      
서울 강남   100
   마포    80
부산 서면    95
   해운대   85
=====내림차순 정렬======
         매출
지역 지점      
서울 마포    80
   강남   100
부산 해운대   85
   서면    95
===값 기준 정렬 (sort_values())===
         매출
지역 지점      
서울 강남   100
부산 서면    95
   해운대   85
서울 마포    80
===인덱스를 컬럼으로 풀고 정렬 후 다시 멀티인덱스로===
         매출
지역 지점      
서울 강남   100
부산 서면    95
   해운대   85
서울 마포    80


#⑤ 데이터프레임 합치기
데이터프레임을 합치거나 연결할 때 사용하는 함수와 메서드로  concat(), merge(), join() 등이 있다.

| 함수명        | 기본 용도          | SQL 방식 유사도          | 기준 키 필요 여부    | 축 방향 선택    | 주요 옵션                                |
| ---------- | -------------- | ------------------- | ------------- | ---------- | ------------------------------------ |
| `concat()` | 위/아래/옆으로 단순 결합 | UNION/UNION ALL     |  없음 (인덱스 기준) | `axis=0/1` | `keys`, `ignore_index` 등             |
| `merge()`  | 공통 컬럼/키로 병합    | JOIN (INNER/LEFT 등) | O 필요          | X          | `on`, `how`, `left_on`, `right_on` 등 |
| `join()`   | 인덱스를 기준으로 병합   | JOIN                | 보통 X (인덱스 기준) | X         | `how`, `on` 등 (left 기준)              |


| 상황                 | 추천 함수      | 이유                |
| ------------------ | ---------- | ----------------- |
| 같은 컬럼 구조, 단순 이어붙이기 | `concat()` | 가장 직관적            |
| SQL처럼 키를 기준으로 병합   | `merge()`  | `on`, `how` 등 유연함 |
| 인덱스를 기준으로 병합       | `join()`   | 인덱스 기반 병합에 최적     |



## pd.concat() – 단순 이어붙이기

In [ ]:
# 위아래로 결합 (행 기준)

import pandas as pd

df1 = pd.DataFrame({'이름': ['철수', '영희'], '점수': [90, 85]})
df2 = pd.DataFrame({'이름': ['민수', '지현'], '점수': [80, 95]})

result = pd.concat([df1, df2], ignore_index=True)
print(result)

# 옆으로 붙이기 (열 기준)
result = pd.concat([df1, df2], axis=1)
print(result)

   이름  점수
0  철수  90
1  영희  85
2  민수  80
3  지현  95
   이름  점수  이름  점수
0  철수  90  민수  80
1  영희  85  지현  95


In [ ]:
df1 = pd.DataFrame({'학번': [1, 2, 3], '이름': ['철수', '영희', '민수']})
df2 = pd.DataFrame({'학번': [2, 3, 4], '점수': [85, 90, 95]})

# 공통 키 '학번' 기준 병합 (inner join)
result = pd.merge(df1, df2, on='학번', how='inner')
result

,학번,이름,점수
0,2,영희,85
1,3,민수,90


# df.join() – 인덱스 기반 병합 (merge의 축소형)

In [ ]:
df1 = pd.DataFrame({'이름': ['철수', '영희', '민수']}, index=[1, 2, 3])
df2 = pd.DataFrame({'점수': [90, 85, 80]}, index=[1, 2, 4])

result = df1.join(df2, how='inner')  # 인덱스 기준 병합
print(result)

#컬럼 키 기준으로 병합도 가능
df1 = pd.DataFrame({'이름': ['철수', '영희', '민수']}, index=[1, 2, 3])
df2 = pd.DataFrame({'이름': ['철수', '영희', '지훈'], '점수': [90, 85, 75]})

# 이름을 기준으로 병합
result = df1.join(df2.set_index('이름'), on='이름') # default => how: MergeHow='left'
print(result)

   이름  점수
1  철수  90
2  영희  85
   이름    점수
1  철수  90.0
2  영희  85.0
3  민수   NaN


## ⑥ 스택(stack)과 언스택(unstack)

- Pandas에서 데이터프레임의 인덱스와 컬럼을 서로 전환할 때 사용하는  함수이다.

| 함수명         | 설명                | 결과 형태                   |
| ----------- | ----------------- | ----------------------- |
| `stack()`   | **컬럼 → 인덱스**로 이동  | 세로로 긴 데이터 (long format) |
| `unstack()` | **인덱스 → 컬럼**으로 이동 | 넓은 데이터 (wide format)    |


In [ ]:
# 데이터 생성

import pandas as pd

data = {
    '과목1': [80, 90],
    '과목2': [85, 95]
}
df = pd.DataFrame(data, index=['철수', '영희'])
df

,과목1,과목2
철수,80,85
영희,90,95


### stack() 사용 – 컬럼(열) =>  인덱스(행)로 변환

In [ ]:
df_stacked = df.stack()
print(type(df_stacked))
print(df_stacked.index)
df_stacked

#결과는 Series이고, 인덱스가 MultiIndex (이름, 과목) 형태로 구성됨

<class 'pandas.core.series.Series'>
MultiIndex([('철수', '과목1'),
            ('철수', '과목2'),
            ('영희', '과목1'),
            ('영희', '과목2')],
           )


철수  과목1    80
    과목2    85
영희  과목1    90
    과목2    95
dtype: int64

### unstack() 사용 – 인덱스(행) => 컬럼(열)으로 변환

In [ ]:
df_unstacked = df_stacked.unstack()
print(df_unstacked)
# 원래 형태로 복원

    과목1  과목2
철수   80   85
영희   90   95


### unstack(level=...)로 특정 인덱스 수준만 이동

In [ ]:
df_stacked.unstack(level=1)  # 과목이 열로 간다
df_stacked
df_stacked.unstack(level=0)  # 이름이 열로 간다
df_stacked

철수  과목1    80
    과목2    85
영희  과목1    90
    과목2    95
dtype: int64

In [ ]:
# 데이터 생성

import pandas as pd

df = pd.DataFrame({
    '학생': ['철수', '영희', '철수', '영희'],
    '과목': ['수학', '수학', '영어', '영어'],
    '점수': [90, 85, 95, 92]
})

# pivot()
pivoted = df.pivot(index='학생', columns='과목', values='점수')
pivoted

과목,수학,영어
학생,,
영희,85,92
철수,90,95


In [ ]:
# pivot_table() 사용 예 (중복 허용 + 집계)
pivoted = df.pivot_table(index='학생', columns='과목', values='점수', aggfunc='mean')
print(pivoted)

# aggfunc='mean': 여러 값이 있으면 평균
# 기본 집계함수는 mean


# 여러 집계함수 적용
df.pivot_table(index='학생', columns='과목', values='점수', aggfunc=['mean', 'max'])


# NaN 처리 옵션
df.pivot_table(index='학생', columns='과목', values='점수', fill_value=0)


과목    수학    영어
학생            
영희  85.0  92.0
철수  90.0  95.0


과목,수학,영어
학생,,
영희,85.0,92.0
철수,90.0,95.0
